In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm_api
import statsmodels.formula.api as smf_api

from warnings import filterwarnings
from sklearn import linear_model

In [26]:
training_data = pd.read_csv("Data/Working/training.csv")
training_data.head()

,date,home_team,away_team,home_score,away_score,home_team_victory,draw,tournament_name,city,country,neutral,scoring_team,scoring_player,minute,own_goal,penalty,shootout_required,first_shooter,winner
0,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Knut Kroon,7.0,False,False,False,NaN,NaN
1,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Lennart Bunke,10.0,False,False,False,NaN,NaN
2,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Bertil Ericsson,13.0,False,False,False,NaN,NaN
3,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Torsten Bunke,43.0,False,False,False,NaN,NaN
4,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Estonia,Leonhard Kass,47.0,False,False,False,NaN,NaN


In [12]:
model = linear_model.LinearRegression()
model.fit(training_data[["home_team_victory"]], training_data[["home_score", "away_score"]])
print(model.intercept_)
print(model.coef_)

[0.88575046 2.74695627]
[[ 3.18910829]
 [-2.18971571]]


In [32]:
"""Linear regression - least square using only home_score and away_score"""
response_data = training_data[["home_team_victory"]]
explanitory_data = training_data[["home_score", "away_score"]]

explanitory_data = sm_api.add_constant(explanitory_data)
model = sm_api.OLS(response_data, explanitory_data)
model_results = model.fit()
print(model_results.summary().tables[1])
print(model_results.aic)


                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5419      0.004    126.761      0.000       0.534       0.550
home_score     0.0773      0.001     86.874      0.000       0.076       0.079
away_score    -0.1217      0.001    -91.576      0.000      -0.124      -0.119
16931.514165044406


In [35]:
model = smf_api.glm(formula="home_team_victory ~ home_score + away_score", data=training_data, family=sm_api.families.Binomial()).fit()
intercept, home_score, away_score = model.params

print(model.summary())
print("home_score, adjusted for away_score:", round(np.exp(home_score), 2))
print("away_score, adjusted for home_score:", round(np.exp(away_score), 2))
print(model.aic)

                                    Generalized Linear Model Regression Results                                    
Dep. Variable:     ['home_team_victory[False]', 'home_team_victory[True]']   No. Observations:                23719
Model:                                                                 GLM   Df Residuals:                    23716
Model Family:                                                     Binomial   Df Model:                            2
Link Function:                                                       Logit   Scale:                          1.0000
Method:                                                               IRLS   Log-Likelihood:                    nan
Date:                                                     Sat, 08 Feb 2025   Deviance:                       110.07
Time:                                                             19:06:21   Pearson chi2:                 3.02e+03
No. Iterations:                                                         

/home/poppop/.local/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/poppop/.local/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: divide by zero encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
/home/poppop/.local/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
